In [1]:
from datetime import date
import pandas as pd
from time import time
import re
import os
from pathlib import Path
import sys

try:
    get_ipython
    from tqdm import tqdm_notebook as tqdm
    %load_ext autoreload
    %autoreload 2
except NameError:
    from tqdm import tqdm

cwd = os.getcwd()
parent = str(Path(cwd).parents[0])
sys.path.append(parent)

from io import StringIO
import csv
import json

In [7]:
def get_data(path):
    with open(path, 'r') as ctgov:
        lines = ctgov.readlines()
        return lines

def text_or_none(data, keys):
    for k in keys:
        try:
            data = data[k]
        except KeyError:
            return None
    return data

def variable_levels(data, level):  
    try:
        return data[level]['text']
    except TypeError:
        try: 
            return data[level]
        except KeyError:
            return None
    except KeyError:
        try: 
            return data[level]
        except KeyError:
            return None

def make_row(jcs):
    td={}
    td['nct_id'] = text_or_none(jcs, ['id_info','nct_id'])
    td['enrollment'] = variable_levels(jcs, 'enrollment')
    return td

def make_df(lines, headers):
    output = StringIO()
    writer = csv.DictWriter(output, fieldnames=headers)
    writer.writeheader()
    for line in lines:
        j = json.loads(line)
        jcs = j['clinical_study']
        td = make_row(jcs)
        writer.writerow(td)
    output.seek(0)
    df = pd.read_csv(output)
    return df

In [8]:
headers = ['nct_id', 'enrollment']

lines = get_data('C:/Users/ndevito/Desktop/FDAAA Implementation Data/Raw JSON/clinicaltrials_raw_clincialtrials_json_2019-09-16.csv')

df = make_df(tqdm(lines), headers)

In [9]:
df.head()

,nct_id,enrollment
0,NCT00000102,NaN
1,NCT00000122,NaN
2,NCT00000126,NaN
3,NCT00000143,61.0
4,NCT00000175,NaN


In [10]:
processed_data = pd.read_csv('C:/Users/ndevito/Dropbox/Python projects/FDAAA Projects/FDAAA Trends Paper/Processed CSVs/applicable_trials_2019-09-16.csv')

In [27]:
due_cohort = processed_data[processed_data.results_due == 1]

In [28]:
enrollments = pd.DataFrame(due_cohort["nct_id"]).merge(df, on = 'nct_id', how = 'left')
enrollments

,nct_id,enrollment
0,NCT00003270,20.0
1,NCT00004088,77.0
2,NCT00074490,442.0
3,NCT00397813,77.0
4,NCT00414076,9.0
...,...,...
4204,NCT03588559,88.0
4205,NCT03641482,13.0
4206,NCT03770871,88.0
4207,NCT03838068,33.0


In [29]:
final_df = enrollments[enrollments.enrollment.notnull()].reset_index(drop=True)
len(final_df)
#only 1 trial without an enrollment number

4208

In [30]:
final_df.enrollment.quantile([0.25, 0.5, 0.75])

0.25     24.0
0.50     57.0
0.75    150.0
Name: enrollment, dtype: float64